<a href="https://colab.research.google.com/github/edurso/tfod-wkspc/blob/master/poke_at_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p /tf/data/
from google.colab import drive
drive.mount('/tf/data')

In [ ]:
import tensorflow as tf
print(tf.__version__)

MODEL = '/tf/data/MyDrive/tensorflow/power-port-targeting/exported-models/trained_model/saved_model'

In [ ]:
from tensorflow.saved_model import load
model = load(MODEL)
model

In [ ]:
print(list(model.signatures.keys()))

In [ ]:
infer = model.signatures['serving_default']
print('inputs: {}\noutputs: {}'.format(infer.inputs, infer.outputs))

In [ ]:
!saved_model_cli show --all --dir /tf/data/MyDrive/tensorflow/power-port-targeting/exported-models/trained_model/saved_model

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv
%matplotlib inline

In [ ]:
img=cv.imread('/tf/data/MyDrive/tensorflow/power-port-targeting/images/test/frame-321.jpg')
x=np.array(img)
plt.imshow(x)

In [ ]:
def extract_tensors(signature_def, graph):
    output = dict()
    for key in signature_def:
        value = signature_def[key]
        if isinstance(value, tf.TensorInfo):
            output[key] = graph.get_tensor_by_name(value.name)
    return output

def extract_tags(signature_def, graph):
    output = dict()
    for key in signature_def:
        output[key] = dict()
        output[key]['inputs'] = extract_tensors(
            signature_def[key].inputs, graph)
        output[key]['outputs'] = extract_tensors(
            signature_def[key].outputs, graph)
    return output

with tf.Session(graph=tf.Graph()) as session:
    serve = tf.saved_model.load(session, tags=['serve'], export_dir=MODEL)
    tags = extract_tags(serve.signature_def, session.graph)
    model = tags['serving_default']
    print(model(img))